### Transformación de la base de datos para poderla trabajar


In [15]:
import pandas
import numpy
import locale
from dateutil.parser import parse
import dtale

In [16]:
# Cargamos el csv con los datos finales
datos = pandas.read_csv("Final_data.csv")

datos['Date'] = datos['Date'].apply(lambda x: x.replace('abri', 'abril'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('gosto', 'agosto'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('dciembre', 'diciembre'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('dicienbre', 'diciembre'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('abrill', 'abril'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('aagosto', 'agosto'))
datos['Date'] = datos['Date'].apply(lambda x: x.replace('Aagosto', 'agosto'))

# eliminar el día de la semana de las cadenas de fecha
datos['Date_2'] = datos['Date'].str.replace(r'^\w+\s+', '')

# convertir la columna de fecha en datetime
datos['Date_2'] = pandas.to_datetime(datos['Date_2'], format='%d de %B de %Y', errors='coerce')


fechas = datos['Date'].unique()



C:\Users\Jorge De León\AppData\Local\Temp\ipykernel_7176\2568158253.py:13: FutureWarning:

The default value of regex will change from True to False in a future version.



In [17]:
locale.setlocale(locale.LC_TIME, 'es_CO.UTF-8')

#datos['Date-2'] = pandas.to_datetime(datos['Date'], format='%A %d de %B de %Y', errors='coerce')

fechas = pandas.DataFrame(fechas)
fechas['date'] = fechas[0].str.replace(r'^\w+\s+', '')

# convertir la columna de fecha en datetime
fechas['date'] = pandas.to_datetime(fechas['date'], format='%d de %B de %Y', errors='coerce')

fechas_malas_2 = fechas.loc[fechas['date'].isna()]


C:\Users\Jorge De León\AppData\Local\Temp\ipykernel_7176\1537350283.py:6: FutureWarning:

The default value of regex will change from True to False in a future version.



In [18]:
datos_finale = datos.drop(['Date'], axis=1)

In [19]:

start_date = datos_finale['Date_2'].min()
end_date = datos_finale['Date_2'].max()
date_range = pandas.date_range(start_date, end_date, freq='D')

categories = datos_finale['Precio $/Kg'].unique()
categories_df = pandas.DataFrame({'Precio $/Kg': categories})

categories_df['key'] = 1
date_range_df = pandas.DataFrame({'key': 1, 'Date_2': date_range})
complete_df = categories_df.merge(date_range_df, on='key').drop('key', axis=1)

df_complete = complete_df.merge(datos_finale, on=['Precio $/Kg', 'Date_2'], how='left')


df_complete = df_complete.sort_values(['Precio $/Kg', 'Date_2'])


df_complete = df_complete.reset_index(drop=True)

columns_to_fill = datos_finale.columns.difference(['Precio $/Kg', 'Date_2'])
df_complete[columns_to_fill] = df_complete.groupby('Precio $/Kg')[columns_to_fill].fillna(value=pandas.NA)

In [20]:
FINAL = df_complete
FINAL = FINAL.drop(['Unnamed: 0'], axis = 1)
mask = FINAL.duplicated(keep='first')
FINAL = FINAL[~mask]

#FINAL = FINAL.applymap(lambda x: pandas.to_numeric(x, errors='coerce'))
excepto = ['Precio $/Kg', 'Date_2']

FINAL[FINAL.columns.difference(excepto)] = FINAL[FINAL.columns.difference(excepto)].apply(lambda x: pandas.to_numeric(x, errors='coerce'))

FINAL.to_csv("Final_database_2.csv")

In [21]:
na_cuenta = FINAL.isna().sum()
na_cuenta = pandas.DataFrame(na_cuenta)
na_cuenta.to_excel("revision_2.xlsx")
print(na_cuenta)

                                0
Precio $/Kg                     0
Date_2                          0
Barranquilla                85060
Bogotá                      59216
Cartagena, Bazurto          90483
Medellín                    57526
Monteria                    87805
Pereira                     73422
Popayán                     93282
Santa Marta                 92814
Sincelejo                   89504
Valledupar                  89375
Villavicencio, CAV          82362
Armenia, Mercar             81194
Cali                        84361
Manizales                   90826
Pasto                       95762
Bucaramanga, Centroabastos  79760
Cúcuta, Cenabastos          80383
Ibagué                      91310
Neiva, Surabastos           85664
Tunja                       80713


In [22]:
revision = pandas.read_excel("revision.xlsx")

Cali_revision = revision.loc[[14,18,28,41], 'Unnamed: 0']
ibague_revision = revision.loc[[20,29,31,36,37,38,46], 'Unnamed: 0']
Medellin_revision = revision.loc[[5,42,49], 'Unnamed: 0']
Pasto_revision = revision.loc[[39,25,16,33,32], 'Unnamed: 0']
Pereira_revision = revision.loc[[7,47, 26, 43], 'Unnamed: 0']
Bogota_revision = revision.loc[[3,48], 'Unnamed: 0']
Manizales_revision = revision.loc[[15,40,24,30], 'Unnamed: 0']
Monteria_revision = revision.loc[[6,44,34], 'Unnamed: 0']
Popayán_revision = revision.loc[[8,45,35], 'Unnamed: 0']
Santa_revision = revision.loc[[9,50], 'Unnamed: 0']
Sincelejo_revision = revision.loc[[10,51], 'Unnamed: 0']
Tunja_revision = revision.loc[[23,27], 'Unnamed: 0']

In [23]:
columnas = na_cuenta.iloc[2:,0]
columnas_2 = columnas.index.to_list()

dataframes_filtered = {}

# Filter the DataFrame for each column
for column in columnas_2:
    df_filtered = FINAL[['Precio $/Kg', 'Date_2', column]]
    dataframes_filtered[column] = df_filtered